In [1]:
from llm.llm.chatgpt import ChatGPT
from llm.llm.gemini import Gemini
from setup_db import create_chroma_db, connect_to_db, execute_query, DBHUB
from llm_test import reasoning_text2SQL, CoT_reasoning
from llm_general import  find_suitable_row_v2

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = Gemini()

In [3]:
question = "For Vietcombank and Techcombank, please provide the quarterly data for the Debt-to-Equity ratio from 2020 to 2023"

In [4]:
find_suitable_row_v2(llm, question, verbose=True)

2024-11-07 19:57:11,737 - INFO - Completion time of gemini-1.5-flash-002: 1.0114305019378662s


prompt_token_count: 197
candidates_token_count: 51
total_token_count: 248

Find suitable column response: 
```json
{
  "industry": ["Banking"],
  "financial_statement_row": ["Total Liabilities", "Total Equity"],
  "financial_ratio_row": ["Debt-to-Equity Ratio"]
}
```



{'industry': ['Banking'],
 'financial_statement_row': ['Total Liabilities', 'Total Equity'],
 'financial_ratio_row': ['Debt-to-Equity Ratio']}

In [5]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [6]:
conn = connect_to_db(db_name, user, password, host, port)


Connecting to database test_db, postgres...


In [7]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_sec_chroma'
persist_directory = 'data/category_sec_chroma'
sec_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_ratio_chroma'
persist_directory = 'data/category_ratio_chroma'
ratio_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'company_name_chroma'
persist_directory = 'data/company_name_chroma'
vector_db_company = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'sql_query'
persist_directory = 'data/sql_query'
vector_db_sql = create_chroma_db(collection_chromadb, persist_directory)

2024-11-07 19:57:12,372 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 19:57:12,494 - INFO - Collection category_bank_chroma is not created.
2024-11-07 19:57:12,978 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 19:57:12,985 - INFO - Collection category_non_bank_chroma is not created.
2024-11-07 19:57:13,491 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 19:57:13,498 - INFO - Collection category_sec_chroma is not created.
2024-11-07 19:57:14,010 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 19:57:14,017 - INFO - Collection category_ratio_chroma is not created.
2024-11-07 19:57:14,542 - INFO - Anonymized telemetry enabled. See                     h

In [8]:
db = DBHUB(conn, bank_vector_store, none_bank_vector_store, sec_vector_store, ratio_vector_store, vector_db_company, vector_db_sql)

In [9]:
prompt = "Compare the Return on Assets (ROA) and Return on Equity (ROE) of Vinamilk and Masan Group for the fiscal year 2023.  Additionally, provide the total assets and total equity for both companies for the same period."

In [10]:
find_suitable_row_v2(llm, prompt, verbose=True)

2024-11-07 19:57:16,019 - INFO - Completion time of gemini-1.5-flash-002: 0.9419925212860107s


prompt_token_count: 215
candidates_token_count: 69
total_token_count: 284

Find suitable column response: 
```json
{
  "industry": ["Food and Beverage", "Consumer Staples"],
  "financial_statement_row": ["Total Assets", "Total Equity", "Net Income"],
  "financial_ratio_row": ["Return on Assets (ROA)", "Return on Equity (ROE)"]
}
```



{'industry': ['Food and Beverage', 'Consumer Staples'],
 'financial_statement_row': ['Total Assets', 'Total Equity', 'Net Income'],
 'financial_ratio_row': ['Return on Assets (ROA)', 'Return on Equity (ROE)']}

In [11]:
from llm_general import *
import time

In [12]:
get_stock_code_based_on_company_name(llm, prompt, db, verbose=True)

2024-11-07 19:57:16,670 - INFO - Completion time of gemini-1.5-flash-002: 0.6346256732940674s


prompt_token_count: 144
candidates_token_count: 26
total_token_count: 170

Get stock code based on company name response: 
```json
{
  "company_name": ["Vinamilk", "Masan Group"]
}
```



2024-11-07 19:57:17,580 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 19:57:18,101 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


(  stock_code                    company_name  \
 0        VNM    Công ty Cổ phần Sữa Việt Nam   
 1        MSN  Công ty Cổ phần Tập đoàn Masan   
 
                               en_company_name            industry  is_bank  \
 0  Vietnam Dairy Products Joint Stock Company  Food and Beverages    False   
 1                     Masan Group Corporation  Food and Beverages    False   
 
    is_securities  
 0          False  
 1          False  ,
 [])

In [20]:
llm.reset_token()
llm.usage()

{'input_token': 0, 'output_token': 0}

In [21]:
start = time.time()
history, error_messages, execution_tables = reasoning_text2SQL(llm, prompt, db, 5, verbose=False)
end = time.time()
print("Time taken: ", end - start)

2024-11-07 19:57:49,718 - INFO - Completion time of gemini-1.5-flash-002: 0.8113832473754883s


prompt_token_count: 206
candidates_token_count: 74
total_token_count: 280



2024-11-07 19:57:50,888 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 19:57:51,252 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 19:57:51,742 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 19:57:52,294 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 19:57:53,020 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 19:57:56,396 - INFO - Completion time of gemini-1.5-flash-002: 3.3688552379608154s


prompt_token_count: 2568
candidates_token_count: 428
total_token_count: 2996

Time taken:  7.504451513290405


In [24]:
for message in history:
    print("==============")
    print(message['role'])
    print('_____________')
    print(message['content'])

system
_____________

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query.
    
user
_____________
You have the following database schema:
You're given database about the financial reports of top Vietnamese companies, both bank and cooperates.

All the data in the financial report are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial reports, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).



Here are the descriptions of tables in the database:



### PostgreSQL tables, with their properties

```sql 

-- Table: company_info

CREATE TABLE company_info(

    stock_code VARCHAR(255) primary key, --The trading symbol.

    is_bank BOOLEAN, --Bool checking wh

In [23]:
last_message = history[-1]['content']
print(last_message)

```sql
SELECT
  ci.stock_code,
  ci.en_company_name,
  fr.data AS roa,
  fr2.data AS roe,
  fa.data AS total_assets,
  fe.data AS total_equity
FROM
  company_info ci
JOIN
  financial_ratio fr ON ci.stock_code = fr.stock_code
JOIN
  financial_ratio fr2 ON ci.stock_code = fr2.stock_code
JOIN
  non_bank_financial_report fa ON ci.stock_code = fa.stock_code
JOIN
  non_bank_financial_report fe ON ci.stock_code = fe.stock_code
JOIN map_category_code_ratio mcr ON fr.ratio_code = mcr.ratio_code
JOIN map_category_code_ratio mcr2 ON fr2.ratio_code = mcr2.ratio_code
JOIN map_category_code_non_bank mcn ON fa.category_code = mcn.category_code
JOIN map_category_code_non_bank mcn2 ON fe.category_code = mcn2.category_code

WHERE
  ci.stock_code IN ('VNM', 'MSN')
  AND fr.year = 2023
  AND fr.quarter = 0
  AND fr2.year = 2023
  AND fr2.quarter = 0
  AND fa.year = 2023
  AND fa.quarter = 0
  AND fe.year = 2023
  AND fe.quarter = 0
  AND mcr.ratio_name = 'Return On Assets'
  AND mcr2.ratio_name = 'Return 

In [17]:
for message in history:
    print("==============")
    print(message['role'])
    print('_____________')
    print(message['content'])

system
_____________

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query.
    
user
_____________
You have the following database schema:
You're given database about the financial reports of top Vietnamese companies, both bank and cooperates.

All the data in the financial report are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial reports, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).



Here are the descriptions of tables in the database:



### PostgreSQL tables, with their properties

```sql 

-- Table: company_info

CREATE TABLE company_info(

    stock_code VARCHAR(255) primary key, --The trading symbol.

    is_bank BOOLEAN, --Bool checking wh

In [18]:
llm.usage()

{'input_token': 2774, 'output_token': 535}

In [19]:
execution_tables[-1]

,stock_code,en_company_name,roa,roe,total_assets,total_equity
